In [ ]:
# 필요한 패키지 불러오지
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_1 = pd.read_csv('book.csv', encoding='euc-kr')

# df_1.head()

In [ ]:
df_1.tail()

In [ ]:
df_1 = df_1.dropna(subset=['Title'])
df_1 = df_1[df_1['Title'].str.strip() != '']

df_2 = df_1.dropna(subset=['Publisher'])
df_2 = df_2[df_2['Publisher'].str.strip() != '']


In [ ]:
# 한국어 타이틀 레코드만 가져옴
df = df_2[df_1['Title'].str.contains('[가-힣]', regex=True)]
# 한국어 [가-힣]
# 영어[A-Za-z]
# 일본어 [ぁ-んァ-ン]


In [ ]:
# price  object => 수치형
df['Price'] = pd.to_numeric(df['Price'].str.replace('[^\d.]', ''), errors='coerce')

# 'Pdate' 컬럼을 날짜 타입으로 변환 (한국어 날짜 형식에 맞춤)
df['Pdate'] = pd.to_datetime(df['Pdate'], format='%Y년 %m월 %d일', errors='coerce')


In [ ]:
# 클러스터링을 위한 패키지
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [ ]:
# 벡터화
df['t_p'] = df['Publisher'] + "_" + df['Title']

tfidf_vectorizer = TfidfVectorizer(max_features=100) # 전체 단어 집합에서 TF-IDF 값이 가장 높은 상위 n개의 단어만을 선택하여 특성 벡터를 생성
X_tfidf = tfidf_vectorizer.fit_transform(df['t_p'])

In [ ]:
# 클러스터링 수행
kmeans = KMeans(n_clusters=20, random_state=42)
kmeans.fit(X_tfidf)

In [ ]:
# 결과 할당
df['Cluster']=kmeans.labels_

In [ ]:
# 클러스터 별로 데이터 확인
for cluster in range(20): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['Title'], '\n') # 각 클러스터에 속한 책 제목 예시 출력

In [ ]:
# 클러스터 결과 시각화
# T-SNE  차원 축소 방법 사용
# 고차원의 TF-IDF 벡터를 2차원으로 매핑한 후 클러스터링 결과를 시각화
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_tfidf.toarray())
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=kmeans.labels_)
plt.show()

Word2Vec 모델

In [ ]:
df['t_p'] = df['Title']
# df['t_p'] = df['Publisher'] + "_" + df['Title']

In [ ]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

# 텍스트 전처리

# 문서를 단어 리스트로 변환
documents = [text.split() for text in df['t_p']]


In [ ]:
documents[0]

In [ ]:
# Word2Vec: 단어의 벡터 변환 모델 중 하나, 비슷한 의미를 가진 단어들이 벡터 공간에서 서로 가까이 위치하도록 학습
model = Word2Vec(sentences=documents, vector_size=100, window=5, min_count=1, workers=4)

# sentences=documents: 학습에 사용될 데이터(문서) 리스트 포맷(각 문서 내 단어들을 리스트)
# vector_size=100: 생성될 단어 벡터 차원 수
# window=5: 단어 예측을 위해 고려하는 주변 단어의 범위. 5:타겟 단어의 앞뒤로 5개의 단어를 컨텍스트로 사용
# min_count=1: 학습에 포함될 최소 단어 빈도수
# workers=4: 스레드의 수(병렬 처리)


#  문서 벡터 생성(<- 단어 벡터 평균)
def document_vector(word2vec_model, doc):
    vector_size = word2vec_model.vector_size   # 단어 벡터 차원 수

    doc_vector = np.zeros(vector_size) # 문서 벡터(초기값 0) 문서 내 각 단어의 벡터를 더함
    num_words = 0
    for word in doc:
        try:
            doc_vector += word2vec_model.wv[word]
            num_words += 1
        except KeyError: # 문서 내 단어의 수
            continue
    if num_words == 0:
        return np.zeros(vector_size)
    else:
        return doc_vector / num_words  # 각 단어 벡터를 더해서 구해진 doc_vector를 num_words로 나누어 평균 계산 (단어 벡터 평균)
                                       # 클러스터링의 input 으로 사용

# 각 문서 벡터 생성
doc_vectors = np.array([document_vector(model, doc) for doc in documents])


In [ ]:
vector = model.wv['강철왕국']
vector

In [ ]:
# K-평균 클러스터링 모델 생성 및 학습
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(doc_vectors)

# 클러스터 할당 결과
df['Cluster'] = kmeans.labels_


In [ ]:
# 클러스터 별로 데이터 확인
for cluster in range(5): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 예시 출력

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 텍스트 토큰화 및 시퀀스 변환
tokenizer = Tokenizer()
# Tokenizer 객체를 사용하여 텍스트를 토큰화, 각 토큰(단어)에 고유한 정수 인덱스 할당
tokenizer.fit_on_texts(df['Title'])
X_seq = tokenizer.texts_to_sequences(df['Title'])

In [ ]:
################# FastText  #################
from gensim.models import FastText

# 예제 문장들
sentences = [['cat', 'say', 'meow'], ['dog', 'say', 'woof']]

# FastText 모델 훈련
model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 'cat' 단어의 벡터 조회
print(model.wv['cat'])

In [ ]:
################# FastText  #################
from gensim.models import FastText
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# FastText 모델 학습 (로컬 데이터에서)
# sentences = [title.split() for title in df['Title']]
sentences = [title.split() for title in df['Title']]
fasttext_model = FastText(X_seq, vector_size=64, window=5, min_count=1)

# 책 제목 -> 벡터화
def vectorize_w_fasttext(text):
    words = text.split()
    word_vectors = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 제목 벡터화
title_vectors = np.array([vectorize_w_fasttext(title) for title in df['Title']])



In [ ]:
# 유사한 단어
model.wv.most_similar('영어')

In [ ]:
# 두 단어간 유사도 확인
model.wv.similarity('수학', '영어')

In [ ]:
# K-평균 클러스터링 모델 생성 및 학습
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(title_vectors)

# 클러스터 할당 결과
df['Cluster'] = kmeans.labels_

In [ ]:
# 클러스터 별로 데이터 확인
for cluster in range(10): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df[df['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 예시 출력
